In [38]:
import data
import importlib
importlib.reload(data)
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 100%|██████████| 5/5 [00:30<00:00,  6.21s/it]

In [52]:
import variables
import contraintes
import importlib
importlib.reload(variables)
importlib.reload(contraintes)

<module 'contraintes' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\contraintes.py'>

In [40]:
def show_solution():
    print(var_dict)
    for machine in var_dict.keys():
        print(machine)
        # for sillon in var_dict[machine].keys():
            # print(var_dict[machine][sillon].X)


In [53]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)



m.setObjective(0, GRB.MINIMIZE)

m.update()
m.printStats()
m.presolve()
m.printStats()
m.optimize()

print(m.objVal)
show_solution()

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 864.55it/s]


Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...
Adding occupation constrainst for: WPY_DEP...

Statistics for modelUnnamed:
  Linear constraint matrix    : 0 Constrs, 48912 Vars, 0 NZs
  Variable types              : 0 Continuous,
48912 Integer (47832 Binary)
  General constraints         : 95664 Constrs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 0, 0 ]
Presolve added 67098 rows and 40764 columns
Presolve time: 1.53s

Statistics for modelUnnamed:
  Linear constraint matrix    : 0 Constrs, 48912 Vars, 0 NZs
  Variable types              : 0 Continuous,
48912 Integer (47832 Binary)
  General constraints         : 95664 Constrs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 0, 0 ]
Gurobi Optimizer version 10.0.0 build v10.0.0

In [ ]:
import contraintes
importlib.reload(contraintes)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...
Adding occupation constrainst for: WPY_DEP...
